In [3]:
# Open and read transcripts in NLTK
f1=codecs.open('../../../private/brainstorming_viz_data/free_group/teamA_part1_divergent.txt', 'r', encoding='utf-8')
f2=codecs.open('../../../private/brainstorming_viz_data/free_group/teamA_part2_convergent.txt', 'r', encoding='utf-8')
f1Temp = codecs.open('tempASCII', 'w', encoding='ascii', errors='ignore')
f2Temp = codecs.open('tempASCII', 'w', encoding='ascii', errors='ignore')
f1Temp.write('free_group/teamA_part1_divergent_bak.txt')
f2Temp.write('free_group/teamA_part1_convergent_bak.txt')
divergent_raw = f1.read()
convergent_raw = f2.read()
regex = re.compile(".*?\[(.*?)\]")


def removeObservations(test_str):
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

divergent = removeObservations(divergent_raw)
convergent = removeObservations(convergent_raw)

NameError: name 'codecs' is not defined

In [4]:
# Extract Noun phrases using TextBlob
blobDivergent = TextBlob(divergent)
blobConvergent = TextBlob(convergent)
nounDiv = blobDivergent.np_counts
nounConv = blobConvergent.np_counts
dfnp1 = pd.DataFrame(columns=['Noun phrases in Divergent phase', 'Frequencies'])
dfnp2 = pd.DataFrame(columns=['Noun phrases in Convergent phase', 'Frequencies'])
u=1
for i in nounDiv:
    dfnp1.loc[u] = [i, nounDiv[i]]
    u+=1

u=1
for i in nounConv:
    dfnp2.loc[u] = [i, nounConv[i]]
    u+=1
    
print("The total number of noun phrases in the divergent phase:", len(nounDiv))
dfnp1.head(15)

NameError: name 'TextBlob' is not defined

In [5]:
print("The total number of noun phrases in the convergent phase:", len(nounConv))
dfnp2.head(15)

NameError: name 'nounConv' is not defined

In [6]:
# Common noun phrases in both phases
common = {}
notcommon = {}
dfc = pd.DataFrame(columns=["Common noun phrases", "Divergent", "Convergent"])
u=1
for i in nounDiv:
    if i in nounConv:
        common[i] = min(nounDiv[i], nounConv[i])
        notcommon[i] = (nounDiv[i], nounConv[i])
        dfc.loc[u] = [i, nounDiv[i], nounConv[i]]
        u+=1
dfc

NameError: name 'pd' is not defined

In [20]:
# Noun phrases that can be removed:
to_be_removed = [u'okay', u'yeah']
keys_to_delete = []
for key in common.keys():
    if key in to_be_removed:
        keys_to_delete.append(key)
for key in keys_to_delete:
    del common[key] # could not have deleted item from iterable 
                    # from within the loop
ccommon=common.keys()

In [21]:
# Function to count co-occurrences of common noun phrases in sentences
def occurrences(source, terms):
    ALL_sentences=sent_tokenize(source) # creates a list of sentences
    combinations_terms = list(itertools.combinations(terms,2)) # NC2 of terms
    n = len(combinations_terms)
    occurlist = []
    for i in range(1, n):
        for j in ALL_sentences:
            temp = list(combinations_terms[i])
            # Find every combination of term 1 followed by term 2 in
            # term pair (1,2)
            out=re.compile(str(temp[0]) + '(.*?)' + str(temp[1]), 
                           re.DOTALL | re.IGNORECASE).findall(j)
            if out:
                occurlist.append(str(temp[0]) + ' - ' + str(temp[1]))
            # Find every combination of term 2 followed by term 1 in
            # term pair (1,2)
            out2 = re.compile(str(temp[1]) + '(.*?)' + str(temp[0]),
                             re.DOTALL | re.IGNORECASE).findall(j)
            if out2:
                occurlist.append(str(temp[1]) + ' - ' + str(temp[0]))
    occurdict = {}
    # count occurrences of term pairs
    for i in occurlist:
        if i not in occurdict:
            occurdict[i] = 1
        else : 
            occurdict[i] = occurdict[i] + 1
    return occurdict

# Call function to find divergent and convergent terms
Divdict = occurrences(divergent, ccommon)
Convdict = occurrences(convergent, ccommon)

In [2]:
# Function to construct a graph of common noun phrases tied by 
# their co-occurrences in sentences of a transcript
def makegraph(occurrences):
    G = nx.Graph()
    for edgs,wt in occurrences.items():
        edg = edgs.split(' - ')
        G.add_edge(edg[0], edg[1], weight=wt)
        G.add_node(edg[0], label = edg[0])
        G.add_node(edg[1], label=edg[1])
    return G

GDiv = makegraph(Divdict)
GConv = makegraph(Convdict)

NameError: name 'Divdict' is not defined

In [1]:
# Plot graph of common phrases in Team A's divergent phase
pos = nx.graphviz_layout(GDiv, prog="dot")
sstt="Team A's wordnet during divergent phase of brainstorming"
possit = nx.draw_networkx(GDiv, sstt, pos=pos, with_edgewidth=True, withLabels=True,
                      labfs=15, valpha=0.2, ealpha=0.7, labelfont=15)

NameError: name 'nx' is not defined